In [1]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np


import statsmodels.api as sm
import scipy.optimize 
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w

# Import Data

In [4]:
# measured data 
a00, a01 = h.import_desired_data("A", "15T")
temp0 =h.normalize(a01["Temp C"]) 
window_sched = a01["Window Open"]


In [3]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a01["DateTime"], y=a01["Window Open"]))
fig.add_trace(go.Scatter(x=a01["DateTime"], y=h.normalize(a01["Temp C"])))

In [4]:
model = sm.tsa.MarkovAutoregression(h.normalize(a01["Temp C"]), k_regimes=2, order=4, switching_ar=False)

result = model.fit()

/Users/julietnwagwuume-ezeoke/_UILCode/windows/localwindowsenv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning:

invalid value encountered in reduce



In [5]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                         Markov Switching Model Results                         
================================================================================
Dep. Variable:                   Temp C   No. Observations:                  381
Model:             MarkovAutoregression   Log Likelihood                1196.722
Date:                  Wed, 21 Jun 2023   AIC                          -2375.445
Time:                          13:31:00   BIC                          -2339.960
Sample:                               0   HQIC                         -2361.366
                                  - 381                                         
Covariance Type:                 approx                                         
                             Regime 0 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6435      0.076      8.478      0.000       0.495       0.792
                             Regime 1 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6435      0.076      8.478      0.000       0.495       0.792
                           Non-switching parameters                           
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2         0.0001   7.93e-06     13.802      0.000    9.39e-05       0.000
ar.L1          1.8187      0.048     37.515      0.000       1.724       1.914
ar.L2         -0.9246      0.101     -9.164      0.000      -1.122      -0.727
ar.L3          0.2539      0.101      2.516      0.012       0.056       0.452
ar.L4         -0.1551      0.049     -3.197      0.001      -0.250      -0.060
                         Regime transition parameters                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p[0->0]        0.5291   1.07e-09   4.93e+08      0.000       0.529       0.529
p[1->0]        0.4739   7.29e-10    6.5e+08      0.000       0.474       0.474
==============================================================================

Warnings:
[1] Covariance matrix calculated using numerical (complex-step) differentiation.
[2] Covariance matrix is singular or near-singular, with condition number 2.65e+23. Standard errors may be unstable.
"""

In [6]:
markov_guess = h.normalize(result.filtered_marginal_probabilities[0])
markov_zscores = h.calc_zscore(markov_guess)

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a01["DateTime"], y=a01["Window Open"]))
fig.add_trace(go.Scatter(x=a01["DateTime"], y=markov_guess))
fig.add_trace(go.Scatter(x=a01["DateTime"], y=markov_zscores))

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a01["DateTime"], y=a01["Window Open"]))
fig.add_trace(go.Scatter(x=a01["DateTime"], y=h.normalize(result.smoothed_marginal_probabilities[0])))

# Support Vector Machine 


## SVM on simulation and obseravtion

In [9]:
sim_data_path = "../../data/energy_model/230307_and_smoothed.pkl"
sim_data = pd.read_pickle(sim_data_path)
sim_temp = h.normalize(sim_data["Temp C"])

In [94]:
temp0_grad = h.normalize(np.diff(temp0))
sim_grad = h.normalize(np.diff(sim_temp))

In [95]:
# sim_temp.iloc[0] # 0.413650456615652
sim_grad[0] # 0.7851199182560861

0.7851199182560861

In [97]:
# x2 = np.array([temp0.to_numpy(), sim_temp.to_numpy()]).T
x2 = np.array([temp0_grad, sim_grad]).T
print(x2[0]) 

[0.36760489 0.78511992]


In [12]:
# highlight on scatter plot when window actions occur 
shift  = window_sched.shift() != window_sched
action_map = np.zeros(len(temp0))
action_map[temp0.loc[shift].index] = 1

In [98]:
fig = make_subplots(rows=1, cols=2, shared_xaxes=False)

# scattered 
fig.add_trace(go.Scatter(x=x2[:,0], y=x2[:,1], mode='markers', showlegend=True, marker_color=action_map, name="Window Actions", ), row=1, col=2)

fig.update_traces(marker=dict(size=8,opacity=0.8, colorscale="Picnic",
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig['layout']['xaxis2']['title']='Observation'
fig['layout']['yaxis2']['title']='Simulation'


# line plots
fig.add_trace(go.Scatter(x=a01["DateTime"], y=window_sched, name="Window Schedule"), row=1, col=1)
fig.add_trace(go.Scatter(x=a01["DateTime"], y=temp0, name="Observation", mode="markers"), row=1, col=1)
fig.add_trace(go.Scatter(x=a01["DateTime"], y=sim_temp, name="Simulation", mode="markers"), row=1, col=1)

fig['layout']['xaxis']['title']='Date'
fig['layout']['yaxis']['title']='Normalized Values'



fig.show()

In [112]:
clf2 = OneClassSVM().fit(x2)

In [118]:
choices2 = clf2.decision_function(x2)
choices2 = np.array([1 if i >= 0 else 0 for i in choices2])

In [119]:
xx, yy = np.meshgrid(np.linspace(-0.1, 1.4, 150, endpoint=False), np.linspace(-0.1,1.4, 150, endpoint=False))
Z = clf2.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

In [120]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=False)

fig .add_trace(
    go.Contour(
        z=Z,
        dx=0.01,
        x0=-0.1,
        dy=0.01,
        y0=-0.1,
        # contours_coloring='lines',
    ))

# line plots

fig.add_trace(go.Scatter(x=x2[:,0], y=x2[:,1], mode='markers', showlegend=True, marker_color=action_map, name="Window Actions", ), row=1, col=1)

fig.update_traces(marker=dict(size=8,opacity=0.8, colorscale="Picnic",
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig['layout']['xaxis']['title']='Observation'
fig['layout']['yaxis']['title']='Simulation'


fig.show()

In [121]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

fig.add_trace(go.Scatter(
    y=x2[:,0], 

    name="observed temp - predictor variable"
), row = 1, col = 1)

fig.add_trace(go.Scatter(
    y=x2[:,1], 
    name="simulated temp - predictor variable"
), row = 1, col = 1)

fig.add_trace(go.Scatter(
    y=window_sched, 
    mode='lines',
    name="window_sched",
), row = 1, col = 1)


fig.add_trace(go.Scatter(
    y=choices2, 
    mode='markers',
    name="prediction",
), row = 1, col = 1)

In [122]:
print(classification_report(window_sched[0:-1], choices2))

              precision    recall  f1-score   support

         0.0       0.53      0.64      0.58       158
         1.0       0.70      0.60      0.65       226

    accuracy                           0.61       384
   macro avg       0.61      0.62      0.61       384
weighted avg       0.63      0.61      0.62       384



## SVM on observation alone

In [2]:
def split(train_size):
    test_size = 1 - train_size
    return (train_size, test_size) 

In [6]:
# predictions on windows data 
x = temp0.to_numpy()
y = window_sched.to_numpy()
# a, b = split(0.2)
# x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=a, test_size=b, shuffle=False)

In [7]:
clf = OneClassSVM().fit(x.reshape(-1,1))


In [8]:
choices = clf.decision_function(x.reshape(-1,1))
choices = np.array([1 if i >= 0 else 0 for i in choices])
# choices

In [9]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

fig.add_trace(go.Scatter(
    y=x, 
    mode='markers',
    name="observed temp - predictor variable"
), row = 1, col = 1)

fig.add_trace(go.Scatter(
    y=y, 
    mode='lines',
    name="window_sched",
), row = 1, col = 1)


fig.add_trace(go.Scatter(
    y=choices, 
    mode='lines',
    name="prediction",
), row = 1, col = 1)


In [10]:
print(classification_report(y, choices))


              precision    recall  f1-score   support

         0.0       0.46      0.55      0.50       159
         1.0       0.63      0.54      0.58       226

    accuracy                           0.54       385
   macro avg       0.54      0.54      0.54       385
weighted avg       0.56      0.54      0.55       385



In [13]:
cr = classification_report(y, choices, output_dict=True)

In [16]:
cr.keys()

dict_keys(['0.0', '1.0', 'accuracy', 'macro avg', 'weighted avg'])

In [17]:
cr["accuracy"]


0.5428571428571428

In [18]:
cr

{'0.0': {'precision': 0.45549738219895286,
  'recall': 0.5471698113207547,
  'f1-score': 0.4971428571428571,
  'support': 159},
 '1.0': {'precision': 0.6288659793814433,
  'recall': 0.5398230088495575,
  'f1-score': 0.580952380952381,
  'support': 226},
 'accuracy': 0.5428571428571428,
 'macro avg': {'precision': 0.542181680790198,
  'recall': 0.543496410085156,
  'f1-score': 0.539047619047619,
  'support': 385},
 'weighted avg': {'precision': 0.5572670002852979,
  'recall': 0.5428571428571428,
  'f1-score': 0.5463401360544218,
  'support': 385}}